# Обработка пропусков в данных, кодирование категориальных признаков, масштабирование данных.

## Импорт библиотек

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 



## Загрузка данных

In [9]:
# Загрузка датасета в DataFrame
titanic_data = pd.read_csv('train.csv')
titanic_data



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Для выполнения лабораторной работы будем использовать датасет, содержащий данные о пассажирах Титаника https://www.kaggle.com/competitions/titanic/data?select=train.csv.

## 1) Обработка пропусков

In [10]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Пропущенные значения содержатся в столбцах Age, Cabin и Embarked. Колонка Cabin содержит 687 пропущенных значений, что составляет около 77% датасета, а также информация о каюте может быть скоррелирована с другими признаками, такими как класс билета (высокий класс, вероятно, имеет более просторные и комфортабельные каюты), или порт посадки (пассажиры, севшие на борт в разных портах, могут быть размещены в разных каютах). Таким образом, колонка Cabin не является важной для решения задачи предсказания выживания пассажиров в датасете.

In [11]:
titanic_data.drop('Cabin', axis=1, inplace=True)

Для более точного анализа, пропущенные значения в колонке Age заполним на основе данных о классе кабины пассажира.
Необходимо сгруппировать пассажиров по классу кабины и вычислить медианный возраст для каждой группы с помощью метода groupby() и функции median(). Затем мы использовать цикл for для заполнения пропущенных значений возраста медианным возрастом для каждой группы пассажиров.

In [12]:
median_age_by_class = titanic_data.groupby('Pclass')['Age'].median()

# заполнить пропущенные значения возраста медианным возрастом для каждого класса кабины
for pclass in range(1, 4):
    titanic_data.loc[(titanic_data['Age'].isnull()) & (titanic_data['Pclass'] == pclass), 'Age'] = median_age_by_class[pclass]


Пропуски в Embarked заполним модой.

In [13]:
most_frequent_embarked = titanic_data['Embarked'].mode()[0]
titanic_data['Embarked'].fillna(most_frequent_embarked, inplace=True)

Проверим полученный датасет на пропуски.

In [14]:
titanic_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

## 2) Кодирование категориальных признаков

Выполним кодирование категориального признака Embarked, обозначающего класс пассажира. Используем one-hot кодирование.

In [17]:
embarked_dummies = pd.get_dummies(titanic_data['Embarked'], prefix='Embarked')
titanic_data_oh = pd.concat([titanic_data, embarked_dummies], axis=1)
titanic_data_oh.drop('Embarked', axis=1, inplace=True)

In [18]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.4500,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,1,0,0


Для кодирования пола используем Label Encoding.

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
titanic_data_oh['Sex'] = le.fit_transform(titanic_data_oh['Sex'])

In [20]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,1,0,0


## 3) Масштабирование данных

В датасете можно выполнить масштабирование данных с помощью методов MinMaxScaler и StandardScaler из модуля sklearn.preprocessing.

С помощью метода MinMaxScaler приведем значения в колонке Age к диапазону от 0 до 1

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
titanic_data_oh['Age_scaled'] = scaler.fit_transform(titanic_data_oh[['Age']])

In [22]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_scaled
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0,1,0.271174
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,1,0,0,0.472229
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,0.321438
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,0,0,1,0.434531
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0,1,0.434531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,0,0,1,0.334004
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,0,0,1,0.233476
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,0,0,1,0.296306
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,1,0,0,0.321438


Используя метод StandardScaler (на основе Z-оценки) (удаляет среднее значение и масштабирует данные до единичной дисперсии). Это означает, что среднее значение всех значений признака будет равно 0, а стандартное отклонение будет равно 1. Вычисляет среднее значение и стандартное отклонение для каждого признака и применяет следующее преобразование для каждого значения признака: Например, если у нас есть колонка "Fare" со значениями, которые распределены нормально с ненулевым средним и различной дисперсией, мы можем использовать StandardScaler, чтобы привести значения к нулевому среднему и единичной дисперсии.

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
titanic_data_oh['Fare_scaled'] = scaler.fit_transform(titanic_data_oh[['Fare']])


In [27]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_scaled,Fare_scaled
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0,1,0.271174,-0.502445
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,1,0,0,0.472229,0.786845
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,0.321438,-0.488854
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,0,0,1,0.434531,0.420730
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0,1,0.434531,-0.486337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,0,0,1,0.334004,-0.386671
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,0,0,1,0.233476,-0.044381
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,0,0,1,0.296306,-0.176263
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,1,0,0,0.321438,-0.044381
